In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:


# Load CSV
df = pd.read_csv("your_file.csv")

# We need subject-level KL grades. Let's assign each subject a representative grade.
# One simple way: take the max KL grade per subject (could also use majority or avg).
subject_grades = df.groupby("ID")["V00XRKL"].max().reset_index()
subject_grades.columns = ["ID", "Grade"]

# Now we stratify by Grade at the subject level
train_ids, temp_ids, y_train, y_temp = train_test_split(
    subject_grades["ID"], subject_grades["Grade"],
    test_size=0.30,  # 30% for val+test
    stratify=subject_grades["Grade"],
    random_state=42
)

# Split remaining into val and test
val_ids, test_ids, y_val, y_test = train_test_split(
    temp_ids, y_temp,
    test_size=2/3,  # from 30%, keep ~20% test, 10% val
    stratify=y_temp,
    random_state=42
)

# Assign splits back to dataframe
df["split"] = "unused"
df.loc[df["ID"].isin(train_ids), "split"] = "train"
df.loc[df["ID"].isin(val_ids), "split"] = "val"
df.loc[df["ID"].isin(test_ids), "split"] = "test"

# Save to file
df.to_csv("split_dataset.csv", index=False)

print(df["split"].value_counts())
print(df.groupby(["split", "V00XRKL"]).size())
